## Biodiversity Project

**Project steps:**
1) Load and wrangler data
2) Identify main trends in data
3) Exploratory Data Analysis
4) Final insights

**Goal:**

Through data-driven exploration, I aspire to illuminate the connections between speciesand parks, informing possible in-danger species and different categories.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

**Step 1** - Load and wrangler data

In [8]:
observations = pd.read_csv("observations.csv")
species_info = pd.read_csv("species_info.csv")

In [24]:
#observations first look
observations.head()

,scientific_name,park_name,observations
0,Vicia benghalensis,Great Smoky Mountains National Park,68
1,Neovison vison,Great Smoky Mountains National Park,77
2,Prunus subcordata,Yosemite National Park,138
3,Abutilon theophrasti,Bryce National Park,84
4,Githopsis specularioides,Great Smoky Mountains National Park,85


In [38]:
#observations data check
print(observations.dtypes)
print(observations.isnull().sum()) # no null data
print(observations.duplicated().sum()) # we have 15 duplicated rows
observations[["scientific_name", "park_name", "observations"]] = observations.duplicated() == True



scientific_name    object
park_name          object
observations        int64
dtype: object
scientific_name    0
park_name          0
observations       0
dtype: int64
15


ValueError: Columns must be same length as key

In [11]:
#species firs look
species_info.head()

,category,scientific_name,common_names,conservation_status
0,Mammal,Clethrionomys gapperi gapperi,Gapper's Red-Backed Vole,NaN
1,Mammal,Bos bison,"American Bison, Bison",NaN
2,Mammal,Bos taurus,"Aurochs, Aurochs, Domestic Cattle (Feral), Dom...",NaN
3,Mammal,Ovis aries,"Domestic Sheep, Mouflon, Red Sheep, Sheep (Feral)",NaN
4,Mammal,Cervus elaphus,Wapiti Or Elk,NaN
